# Newton's Method

In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

 Activating environment at `~/Dropbox/class/M1399.000200/2020/Project.toml`


Status `~/Dropbox/class/M1399.000200/2020/Project.toml`
  [7d9fca2a] Arpack v0.4.0
  [6e4b80f9] BenchmarkTools v0.5.0
  [1e616198] COSMO v0.7.7
  [f65535da] Convex v0.13.7
  [a93c6f00] DataFrames v0.21.8
  [31c24e10] Distributions v0.23.12
  [e2685f51] ECOS v0.12.1
  [f6369f11] ForwardDiff v0.10.12
  [c91e804a] Gadfly v1.3.1
  [bd48cda9] GraphRecipes v0.5.4
  [2e9cd046] Gurobi v0.9.2
  [82e4d734] ImageIO v0.3.1
  [6218d12a] ImageMagick v1.1.6
  [916415d5] Images v0.23.1
  [b6b21f68] Ipopt v0.6.3
  [42fd0dbc] IterativeSolvers v0.8.4
  [4076af6c] JuMP v0.21.4
  [b51810bb] MatrixDepot v0.9.0-DEV #master (https://github.com/JuliaMatrices/MatrixDepot.jl.git)
  [6405355b] Mosek v1.1.3
  [1ec41992] MosekTools v0.9.4
  [76087f3c] NLopt v0.6.1
  [47be7bcc] ORCA v0.5.0
  [a03496cd] PlotlyBase v0.4.1
  [f0f68f2c] PlotlyJS v0.14.0
  [91a5bcdd] Plots v1.6.12
  [438e738f] PyCall v1.92.1
  [d330b81b] PyPlot v2.9.0
  [dca85d43] QuartzImageIO v0.7.3
  [6f49c342] RCall v0.13.9
  [ce6b1742] RDatasets v0.

### Unconstrained optimization 

* Problem
$$
\min_{\mathbf{x}\in \mathbb{R}^d} f(\mathbf{x})
$$

* First-order optimality condition
$$
\nabla f(\mathbf{x}^\star) = 0.
$$

## Newton's method

* Newton's method was originally developed by, not surprisingly, Isaac Newton, and further refined by [Joseph Raphson](https://en.wikipedia.org/wiki/Joseph_Raphson), to find roots of nonlinear equations
$g(x) = 0$:
$$
    x^{(t+1)} = x^{(t)} - \frac{g(x^{(t)}}{g'(x^{(t)})}.
$$

* In optimization, we may solve the nonlinear equation $\nabla f(\mathbf{x}) = 0$ by Newton-Raphson, i.e., setting $g(\mathbf{x})=\nabla f(\mathbf{x})$.


* Newton's method, a.k.a. **Newton-Raphson method**, is considered the gold standard for its fast (quadratic) convergence
$$
	\frac{\|\mathbf{x}^{(t+1)} - \mathbf{x}^*\|}{\|\mathbf{x}^{(t)} - \mathbf{x}^*\|^2} \to \text{constant}.
$$
In words, the estimate gets accurate by two decimal digits per iteration.

* Idea: iterative quadratic approximation. 

* Second-order Taylor expansion of the objective function around the current iterate $\mathbf{x}^{(t)}$
$$
	f(\mathbf{x}) \approx f(\mathbf{x}^{(t)}) + \nabla f(\mathbf{x}^{(t)})^T (\mathbf{x} - \mathbf{x}^{(t)}) + \frac {1}{2} (\mathbf{x} - \mathbf{x}^{(t)})^T [\nabla^2 f(\mathbf{x}^{(t)})] (\mathbf{x} - \mathbf{x}^{(t)})
$$
and then minimize the quadratic approximation.

* To maximize the quadratic appriximation function, we equate its gradient to zero
$$
	\nabla f(\mathbf{x}^{(t)}) + [\nabla^2 f(\mathbf{x}^{(t)})] (\mathbf{x} - \mathbf{x}^{(t)}) = \mathbf{0},
$$
which suggests the next iterate
$$
\begin{eqnarray*}
	\mathbf{x}^{(t+1)} &=& \mathbf{x}^{(t)} - [\nabla^2 f(\mathbf{x}^{(t)})]^{-1} \nabla f(\mathbf{x}^{(t)}),
\end{eqnarray*}
$$
a complete analogue of the univariate Newton-Raphson for solving $g(x)=0$.

We call this **naive Newton's method**.

* **Stability issue**: naive Newton's iterate is **not** guaranteed to be a descent algorithm, i.e., that ensures $f(\mathbf{x}^{(t+1)} \le f(\mathbf{x}^{(t)})$. It's equally happy to head uphill or downhill. Following example shows that the Newton iterate converges to a local maximum, converges to a local minimum, or diverges depending on starting points.

In [5]:
using Plots; gr()
using ForwardDiff  # for symbolic differentiation

f(x) = sin(x)
df = x -> ForwardDiff.derivative(f, x) # gradient
d2f = x -> ForwardDiff.derivative(df, x) # hessian
xarray = [2.0, 2.75, 4.0] # start point: 2.0 (local maximum), 2.75 (diverge), 4.0 (local minimum)
for i=1:length(xarray)
    x = xarray[i]
    titletext = "Starting point: $x"
    anim = @animate for iter in 0:10
        iter > 0 && (x = x - d2f(x) \ df(x))
        p = Plots.plot(f, 0, 2π, xlim=(0, 2π), ylim=(-1.1, 1.1), legend=nothing, title=titletext)
        Plots.plot!(p, [x], [f(x)], shape=:circle)
        Plots.annotate!(p, x, f(x), text("x($iter)", :right))
    end
    gif(anim, string("./newton_sine_", i, ".gif"), fps = 1);
end

┌ Info: Saved animation to 
│   fn = /Users/jhwon/Dropbox/class/M1399.000200/2020/lectures/22-newton/newton_sine_1.gif
└ @ Plots /Users/jhwon/.julia/packages/Plots/uCh2y/src/animation.jl:104
┌ Info: Saved animation to 
│   fn = /Users/jhwon/Dropbox/class/M1399.000200/2020/lectures/22-newton/newton_sine_2.gif
└ @ Plots /Users/jhwon/.julia/packages/Plots/uCh2y/src/animation.jl:104
┌ Info: Saved animation to 
│   fn = /Users/jhwon/Dropbox/class/M1399.000200/2020/lectures/22-newton/newton_sine_3.gif
└ @ Plots /Users/jhwon/.julia/packages/Plots/uCh2y/src/animation.jl:104


<img src="./newton_sine_1.gif">

<img src="./newton_sine_2.gif">

<img src="./newton_sine_3.gif">

* A remedy for the instability issue:
    1. approximate $\nabla^2 f(\mathbf{x}^{(t)})$ by a positive definite $\mathbf{H}^{(t)}$ (if it's not), **and** 
    2. line search (backtracking) to ensure the descent property.   

* Why insist on a _positive definite_ approximation of Hessian? By first-order Taylor expansion,
$$
\begin{eqnarray*}
    & & f(\mathbf{x}^{(t)} + s \Delta \mathbf{x}^{(t)}) - f(\mathbf{x}^{(t)}) \\
    &=& \nabla f(\mathbf{x}^{(t)})^T s \Delta \mathbf{x}^{(t)} + o(s) \\
    &=& - s \cdot \nabla f(\mathbf{x}^{(t)})^T [\mathbf{H}^{(t)}]^{-1} \nabla f(\mathbf{x}^{(t)}) + o(s).
\end{eqnarray*}
$$
For $s$ sufficiently small, $f(\mathbf{x}^{(t)} + s \Delta \mathbf{x}^{(t)}) - f(\mathbf{x}^{(t)})$ is strictly negative if $\mathbf{H}^{(t)}$ is positive definite. The quantity $\{\nabla f(\mathbf{x}^{(t)})^T [\mathbf{H}^{(t)}]^{-1} \nabla f(\mathbf{x}^{(t)})\}^{1/2}$ is termed the **Newton decrement**.

* In summary, a **practical Newton-type algorithm** iterates according to
$$
	\boxed{ \mathbf{x}^{(t+1)} = \mathbf{x}^{(t)} - s [\mathbf{H}^{(t)}]^{-1} \nabla f(\mathbf{x}^{(t)}) 
	= \mathbf{x}^{(t)} + s \Delta \mathbf{x}^{(t)} }
$$
where $\mathbf{H}^{(t)}$ is a positive definite approximation to $\nabla^2 f(\mathbf{x}^{(t)})$ and $s$ is a step size.

* For strictly convex $f$, $\nabla^2 f(\mathbf{x}^{(t)})$ is always positive definite. In this case, the above algorithm is called **damped Newton**. However, line search is still needed (at least for a finite number of times) to guarantee convergence.

* Line search strategy: step-halving ($s=1,1/2,\ldots$), golden section search, cubic interpolation, Amijo rule, ... Note the **Newton direction**  
$$
\Delta \mathbf{x}^{(t)} = [\mathbf{H}^{(t)}]^{-1} \nabla f(\mathbf{x}^{(t)})
$$
only needs to be calculated once. Cost of line search mainly lies in objective function evaluation.
```Julia
    # Backtracking line search
    # given: descent direction ∆x, x ∈ domf, α ∈ (0,0.5), β ∈ (0,1).
    t = 1.0
    while f(x + t∆x) > f(x) + α * t * ∇f(x)'∆x
        t *= β
    end
```

<img src="https://jfukuyama.github.io/teaching/stat710/notes/backtracking-image.png" width=400>

> The lower dashed line shows the linear extrapolation of $f$, and the upper dashed line has a slope a factor of α smaller. The backtracking condition is that $f$ lies below the upper dashed line, i.e., $0 \le t \le t_0$.  -- Boyd & Vandenberghe Sect 9.2

* How to approximate $\nabla^2 f(\mathbf{x})$? More of an art than science. Often requires problem specific analysis. 

* Taking $\mathbf{H} = \mathbf{I}$ leads to the **gradient descent method**.

<img src="http://trond.hjorteland.com/thesis/img208.gif" width="400" align="center"/>

## Fisher scoring

* Consider MLE in which $f(\mathbf{x}) = -\ell(\boldsymbol{\theta})$, where $\ell(\boldsymbol{\theta})$ is the log-likelihood of parameter $\boldsymbol{\theta}$.

* **Fisher scoring method**: replace $- \nabla^2 \ell(\boldsymbol{\theta})$ by the expected Fisher information matrix
$$
	\mathbf{I}(\theta) = \mathbf{E}[-\nabla^2\ell(\boldsymbol{\theta})] = \mathbf{E}[\nabla \ell(\boldsymbol{\theta}) \nabla \ell(\boldsymbol{\theta})^T] \succeq \mathbf{0},
$$
which is true under exchangeability of tne expectation and the differentiation.

    Therefore we set $\mathbf{H}^{(t)}=\mathbf{I}(\boldsymbol{\theta}^{(t)})$ and obtain the Fisher scoring algorithm: 
$$
	\boxed{ \boldsymbol{\theta}^{(t+1)} = \boldsymbol{\theta}^{(t)} + s [\mathbf{I}(\boldsymbol{\theta}^{(t)})]^{-1} \nabla \ell(\boldsymbol{\theta}^{(t)})}.
$$

* Combined with line search, a descent algorithm can be devised.

### Example: logistic regression

* Binary data: $y_i \in \{0,1\}$, $\mathbf{x}_i \in \mathbb{R}^{p}$. 

* Model: $y_i \sim $Bernoulli$(p_i)$, where
\begin{eqnarray*}
	\mathbf{E} (y_i) = p_i &=& g^{-1}(\eta_i) = \frac{e^{\eta_i}}{1+ e^{\eta_i}} \quad \text{(mean function, inverse link function)} \\
	\eta_i = \mathbf{x}_i^T \beta &=& g(p_i) = \log \left( \frac{p_i}{1-p_i} \right) \quad \text{(logit link function)}.
\end{eqnarray*}


* MLE: density
\begin{eqnarray*}
	f(y_i|p_i) &=& p_i^{y_i} (1-p_i)^{1-y_i} \\
	&=& e^{y_i \log p_i + (1-y_i) \log (1-p_i)} \\
	&=& \exp\left( y_i \log \frac{p_i}{1-p_i} + \log (1-p_i)\right).
\end{eqnarray*}


* Log likelihood of the data $(y_i,\mathbf{x}_i)$, $i=1,\ldots,n$, and its derivatives are

\begin{eqnarray*}
	\ell(\beta) &=& \sum_{i=1}^n \left[ y_i \ln p_i + (1-y_i) \log (1-p_i) \right] \\
	&=& \sum_{i=1}^n \left[ y_i \mathbf{x}_i^T \beta - \log (1 + e^{\mathbf{x}_i^T \beta}) \right] \\
	\nabla \ell(\beta) &=& \sum_{i=1}^n \left( y_i \mathbf{x}_i - \frac{e^{\mathbf{x}_i^T \beta}}{1+e^{\mathbf{x}_i^T \beta}} \mathbf{x}_i \right) \\
	&=& \sum_{i=1}^n (y_i - p_i) \mathbf{x}_i = \mathbf{X}^T (\mathbf{y} - \mathbf{p})	\\
	- \nabla^2\ell(\beta) &=& \sum_{i=1}^n p_i(1-p_i) \mathbf{x}_i \mathbf{x}_i^T = \mathbf{X}^T \mathbf{W} \mathbf{X}, \quad
	\text{where } \mathbf{W} &=& \text{diag}(w_1,\ldots,w_n), w_i = p_i (1-p_i) \\
	\mathbf{I}(\beta) &=& \mathbf{E} [- \nabla^2\ell(\beta)] = \mathbf{X}^T \mathbf{W} \mathbf{X} = - \nabla^2\ell(\beta)
\end{eqnarray*}

(why the last line?)

* Therefore for this problem **Newton's method == Fisher scoring**: 
$$
\begin{eqnarray*}
	\beta^{(t+1)} &=& \beta^{(t)} + s[-\nabla^2 \ell(\beta^{(t)})]^{-1} \nabla \ell(\beta^{(t)})	\\
	&=& \beta^{(t)} + s(\mathbf{X}^T \mathbf{W}^{(t)} \mathbf{X})^{-1} \mathbf{X}^T (\mathbf{y} - \mathbf{p}^{(t)}) \\
	&=& (\mathbf{X}^T \mathbf{W}^{(t)} \mathbf{X})^{-1} \mathbf{X}^T \mathbf{W}^{(t)} \left[ \mathbf{X} \beta^{(t)} + s(\mathbf{W}^{(t)})^{-1} (\mathbf{y} - \mathbf{p}^{(t)}) \right] \\
	&=& (\mathbf{X}^T \mathbf{W}^{(t)} \mathbf{X})^{-1} \mathbf{X}^T \mathbf{W}^{(t)} \mathbf{z}^{(t)},
\end{eqnarray*}
$$
where 
$$
	\mathbf{z}^{(t)} = \mathbf{X} \beta^{(t)} + s(\mathbf{W}^{(t)})^{-1} (\mathbf{y} - \mathbf{p}^{(t)})
$$ 
are the working responses. A Newton iteration is equivalent to solving a weighed least squares problem $\sum_{i=1}^n w_i (z_i - \mathbf{x}_i^T \beta)^2$. Thus the name **IRLS (iteratively re-weighted least squares)**.

* Implication: if a weighted least squares solver is at hand, then logistic regression models can be fitted.
    - Now we know alternatives -- GP solver or exponential cone solver

### Example: GLM 

Let's consider a more general class of generalized linear models (GLM).

#### Exponential families

* Random variable $Y$ belongs to an exponential family if the density
$$
	p(y|\eta,\phi) = \exp \left\{ \frac{y\eta - b(\eta)}{a(\phi)} + c(y,\phi) \right\}.
$$
    * $\eta$: natural parameter.  
    * $\phi>0$: dispersion parameter.  
    * Mean: $\mu= b'(\eta)$. When $b'(\cdot)$ is invertible, function $g(\cdot)=[b']^{-1}(\cdot)$ is called the canonical link function.
    * Variance $\mathbf{Var}{Y}=b''(\eta)a(\phi)$.


* For example, if $Y \sim \text{Ber}(\mu)$, then
$$
p(y|\eta,\phi) = \exp\left( y \log \frac{\mu}{1-\mu} + \log (1-\mu)\right).
$$
Hence
$$
\eta = \log \frac{\mu}{1-\mu}, \quad
\mu = \frac{e^{\eta}}{1+e^{\eta}},
\quad
b(\eta) = -\log (1-\mu) = \log(1+e^{\eta})
$$
Hence
$$
b'(\eta) = \frac{e^{\eta}}{1+e^{\eta}} = g^{-1}(\eta).
$$
as above.


| Family           | Canonical Link                                 | Variance Function |
|------------------|------------------------------------------------|-------------------|
| Normal (unit variance)           | $\eta=\mu$                                     | 1                 |
| Poisson          | $\eta=\log \mu$                                | $\mu$             |
| Binomial         | $\eta=\log \left(\frac{\mu}{1 - \mu} \right)$  | $\mu (1 - \mu)$   |
| Gamma            | $\eta = \mu^{-1}$                              | $\mu^2$           |
| Inverse Gaussian | $\eta = \mu^{-2}$                              | $\mu^3$           |

#### Generalized linear models

GLM models the conditional distribution of $Y$ given predictors $\mathbf{x}$ through
the conditional mean $\mu = \mathbf{E}(Y|\mathbf{x})$ via a strictly increasing link function
$$
	g(\mu) = \mathbf{x}^T \beta, \quad \mu = g^{-1}(\mathbf{x}^T\beta) = b'(\eta)
$$

From these relations we have (assuming no overdispertion, i.e., $a(\phi)\equiv 1$)
$$
\mathbf{x}^T\beta = g'(\mu)d\mu,
\quad
d\mu = b''(\eta)d\eta, 
\quad
b''(\eta) = \mathbf{Var}[Y] = \sigma^2,
\quad
d\eta = \frac{1}{b''(\eta)}d\mu = \frac{1}{b''(\eta)g'(\mu)}\mathbf{x}^T d\beta.
$$

Then
\begin{align*}
d\ell(\beta) &= yd\eta - b'(\eta)d\eta = yd\eta - \mu d\eta = (y-\mu)d\eta \\
    &= \frac{(y-\mu)\mathbf{x}^T}{b''(\eta)g'(\mu)}d\beta 
    = \frac{1}{\sigma^2}\frac{(y-\mu)\mathbf{x}^T}{g'(\mu)}d\beta
    = \nabla\ell(\beta)^Td\beta,
\end{align*}
i.e.,
$$
\nabla\ell(\beta) = \frac{1}{g'(\mu)}\frac{(y-\mu)\mathbf{x}}{\sigma^2},
\quad
\mu = g^{-1}(\mathbf{x}^T\beta).
$$

To compute the Hessian, observe that
\begin{align*}
d^2\eta &= -\frac{(\mathbf{x}^Td\beta)[b'''(\eta)d\eta g'(\mu) + b''(\eta)g''(\mu)d\mu]}{[b''(\eta)g'(\mu)]^2} \\
    &= -\frac{(\mathbf{x}^Td\beta)[b'''(\eta)/b''(\eta)(\mathbf{x}^T d\beta) g'(\mu) + b''(\eta)g''(\mu)/g'(\mu)(\mathbf{x}^Td\beta)]}{[b''(\eta)g'(\mu)]^2} \\
    &= -\frac{b'''(\eta)g'(\mu)+[b''(\eta)]^2g''(\mu)}{[b''(\eta)g'(\mu)]^3}d\beta^T\mathbf{x}\mathbf{x}^Td\beta.
\end{align*}

Then
\begin{align*}
d^2\ell(\beta) &= -d\mu d\eta + (y-\mu)d^2\eta \\
    &= -\frac{b''(\eta)}{[b''(\eta)g'(\mu)]^2}d\beta^T\mathbf{x}\mathbf{x}^Td\beta  
        -(y-\mu)\frac{b'''(\eta)g'(\mu)+[b''(\eta)]^2g''(\mu)}{[b''(\eta)g'(\mu)]^3}d\beta^T\mathbf{x}\mathbf{x}^Td\beta
\end{align*}
yielding
$$
\nabla^2\ell(\beta) = -\frac{1}{b''(\eta)[g'(\mu)]^2}\mathbf{x}\mathbf{x}^T
        -(y-\mu)\frac{b'''(\eta)/[b''(\eta)g'(\mu)]^2+g''(\mu)/[g'(\mu)]^3}{b''(\eta)}\mathbf{x}\mathbf{x}^T.
$$

To sum up, for $n$ samples we have

* Score, Hessian, information

\begin{eqnarray*}
 	\nabla\ell(\beta) &=& \sum_{i=1}^n \frac{(y_i-\mu_i) [1/g'(\mu_i)]}{\sigma^2} \mathbf{x}_i, \quad \mu_i = \mathbf{x}_i^T\beta, \\
	- \nabla^2 \ell(\beta) &=& \sum_{i=1}^n \frac{[1/g'(\mu_i)]^2}{\sigma^2} \mathbf{x}_i \mathbf{x}_i^T - \sum_{i=1}^n \frac{(y_i - \mu_i)[b'''(\eta_i)/[b''(\eta_i)g'(\mu_i)]^2+g''(\mu_i)/[g'(\mu_i)]^3]}{\sigma^2} \mathbf{x}_i \mathbf{x}_i^T, \quad \eta_i = [b']^{-1}(\eta),	\\
	\mathbf{I}(\beta) &=& \mathbf{E} [- \nabla^2 \ell(\beta)] = \sum_{i=1}^n \frac{[1/g'(\mu_i)]^2}{\sigma^2} \mathbf{x}_i \mathbf{x}_i^T = \mathbf{X}^T \mathbf{W} \mathbf{X}.
\end{eqnarray*}    


* Fisher scoring method:
$$
 	\beta^{(t+1)} = \beta^{(t)} + s [\mathbf{I}(\beta^{(t)})]^{-1} \nabla \ell(\beta^{(t)})
$$
IRLS with weights $w_i = [1/g'(\mu_i)]^2/\sigma^2$ and some working responses $z_i$.

* For *canonical link*, $\mathbf{x}^T\beta = g(\mu) =[b']^{-1}(\mu) = \eta$. The second term of Hessian vanishes because $d\eta=\mathbf{x}^Td\beta$ and $d^2\eta=0$. The Hessian coincides with Fisher information matrix. **Fisher scoring == Newton's method**. Hence MLE is a *convex* optimization problem.

 
* Non-canonical link, **Fisher scoring != Newton's method**, and MLE is a *non-convex* optimization problem.

  Example: Probit regression (binary response with probit link).
\begin{eqnarray*}
    y_i &\sim& \text{Ber}(p_i) \\
    p_i &=& \Phi(\mathbf{x}_i^T \beta)  \\
    \eta_i &=& \log\left(\frac{p_i}{1-p_i}\right) \neq \mathbf{x}_i^T \beta = \Phi^{-1}(p_i).
\end{eqnarray*}
  where $\Phi(\cdot)$ is the cdf of a standard normal.
 
* Julia, R, and Matlab implement the Fisher scoring method, aka IRLS, for GLMs.
    * [GLM.jl](https://github.com/JuliaStats/GLM.jl) package.

## Nonlinear regression - Gauss-Newton method

* Now we finally get to the problem Gauss faced in 1801!  
Relocate the dwarf planet Ceres <https://en.wikipedia.org/wiki/Ceres_(dwarf_planet)> by fitting 24 observations to a 6-parameter (nonlinear) orbit.
    - In 1801, Jan 1 -- Feb 11 (41 days), astronomer Piazzi discovered Ceres, which was lost behind the Sun after observing its orbit 24 times.
    - Aug -- Sep, futile search by top astronomers; Laplace claimed it unsolvable.
    - Oct -- Nov, Gauss did calculations by method of least squares, sent his results to astronomer von Zach.
    - Dec 31, von Zach relocated Ceres according to Gauss’ calculation.

* Nonlinear least squares (curve fitting): 
$$
	\text{minimize} \,\, f(\beta) = \frac{1}{2} \sum_{i=1}^n [y_i - \mu_i(\mathbf{x}_i, \beta)]^2
$$
For example, $y_i =$ dry weight of onion and $x_i=$ growth time, and we want to fit a 3-parameter growth curve
$$
	\mu(x, \beta_1,\beta_2,\beta_3) = \frac{\beta_3}{1 + \exp(-\beta_1 - \beta_2 x)}.
$$

<img src="https://cdn.xlstat.com/img/tutorials/nlin5.gif" width="300" align="center"/>

* "Score" and "information matrices"
$$
\begin{eqnarray*}
	\nabla f(\beta) &=& - \sum_{i=1}^n [y_i - \mu_i(\beta)] \nabla \mu_i(\beta) \\
	\nabla^2 f(\beta) &=& \sum_{i=1}^n \nabla \mu_i(\beta) \nabla \mu_i(\beta)^T - \sum_{i=1}^n [y_i - \mu_i(\beta)] \nabla^2 \mu_i(\beta) \\
	\mathbf{I}(\beta) &=& \sum_{i=1}^n \nabla \mu_i(\beta) \nabla \mu_i(\beta)^T = \mathbf{J}(\beta)^T \mathbf{J}(\beta),
\end{eqnarray*}
$$
where $\mathbf{J}(\beta)^T = [\nabla \mu_1(\beta), \ldots, \nabla \mu_n(\beta)] \in \mathbb{R}^{p \times n}$.

* **Gauss-Newton** (= "Fisher scoring method") uses $\mathbf{I}(\beta)$, which is always positive semidefinite.
$$
	\boxed{ \beta^{(t+1)} = \beta^{(t)} - s [\mathbf{I} (\beta^{(t)})]^{-1} \nabla f(\beta^{(t)}) }
$$
based on the rationale that either the residuals $y_i − \mu_i(\beta)$ are small or the regression functions $\mu_i(\beta)$ are nearly linear.

* **Levenberg-Marquardt** method or **damped least squares algorithm (DLS)**, adds a ridge term to the approximate Hessian
$$
	\boxed{ \beta^{(t+1)} = \beta^{(t)} - s [\mathbf{I} (\beta^{(t)}) + \tau \mathbf{Id}_p]^{-1} \nabla f(\beta^{(t)}) }
$$
bridging between Gauss-Newton and steepest descent.

## References

* Boyd and Vandenberghe, 2004. Convex Optimization. Cambridge University Press. (Ch. 9)

* Lange, K., 2010. Numerical analysis for statisticians. Springer Science & Business Media. (Ch. 14)

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.